In [ ]:
from sympy import Point, Line, Segment, Ray, intersection
import functools

# Алгоритм балабана

___Алгоритм Балабана___ - детерминированный алгоритм, позволяющий по множеству отрезков на плоскости получить множество точек, в которых эти отрезки пересекаются.

## Введение

---

Решение задачи по поиску множества пересечений отрезков является одной из главных задач вычислительной геометрии. Рассмотрим несколько самых распространенных алгоритмов:

1. Тривиальный детерминированный алгоритм имеет временную сложность $O(n^2)$, и его суть заключается в проверке попарного пересечения отрезков.
2. Сложнее, но эффективнее алгоритм Бентли-Оттмана с оценкой сложности $O((n + k) \log n +k)$, в основе которого лежит метод заметающей прямой.
3. Алгоритм, предложенный Чазелле и Едельсбруннером, имеет лучшую оценку $O(n \log n + k)$, но в отличие от предыдущих методов требует квадратичной памяти.
4. Оптимальный детерминированный алгоритм был предложен Балабаном с временной оценкой сложности $O(n \log(n) + k)$ и $O(n)$ памяти, где К - число пересекающихся отрезков.

При количестве отрезков от 2000, и большому количеству пересечений целесообразно использовать алгоритм Балабана. Однако в результате громоздкости и высокой сложности реализации алгоритма, в большинстве практических задач используется алгоритм заметающей прямой Бентли-Оттмана.


## Основные понятия

---

Введем некоторые обозначения. Пусть $Int(S)$ - множество всех точек пересечения отрезков из множества $S$, а $K = |Int(S)|$ - количество таких пересечений.<br>
Через $\langle a, b \rangle$ обозначим вертикальную полосу, которая ограничена прямыми $x = a$ и $x = b$, а через $s$ - отрезок с вершинами в точках с абсциссами $l$ и $r$.<br>
Рассмотрим взаимное расположение вертикальной полосы $\langle a, b \rangle$ и отрезка $s$. 

##### Определение
> Будем говорить, что отрезок $s$, с вершинами в точках с абсциссами $l$ и $r$ :
- _охватывает_ (_span_) полосу $\langle a, b \rangle$, если $l \le a \le b \le r$; <br>
- _внутренний_ (_inner_) для полосы $\langle a, b \rangle$, если $a < l < r < b$; <br>
- _пересекает_ (_cross_) полосу $\langle a, b \rangle$ в других случаях.


##### Определение
> Два отрезка $s_1$ и $s_2$ называются _пересекающимися внутри_ полосы $\langle a, b \rangle$, если их точка пересечения лежит в пределах этой полосы.<br>

Для двух множеств отрезков $S$ и $S'$ определим множество $Int(S, S')$ как $\{ {s, s'} | (s \in S, s' \in S') \& (s \ Intersect \ s') \}$.

![pic1](http://images.vfl.ru/ii/1519671635/54468d52/20745922.png)
$D = ((s_1, s_2, s_3), \langle a, b \rangle)$, $Loc(D, s_4) = 0$, $Loc(D, s_5) = 2$ или $3$, $Int(D, \{s_4,\ s_5\}) = \{\{s_3,\ s_5\}\}$

Обозначения $Int_{a, b}(S)$ и $Int_{a, b}(S, S')$ будут использоваться для описания подмножеств $Int(S)$ и $Int(S, S')$, состоящих из пересекающихся пар отрезков в пределах полосы $\langle a, b \rangle$.

Далее фигурные скобки $\{\}$ используются для определения неупорядоченных множеств, а круглые скобки $()$ используются для определения упорядоченных множеств.

##### Определение
> Введем отношение порядка на множестве отрезков $s_1 <_a s_2$ если оба отрезка пересекают вертикальную линию<br> $x = a$ и точка пересечения этой прямой с отрезком $s_1$ лежит ниже точки пересечения с $s_2$.


##### Определение
> ___Лестница___ $D$ — это пара $(Q, \langle a, b \rangle)$, в которой отрезки из множества $Q$ удовлетворяют следующим условиям : 
- любой отрезок из $Q$ охватывает полосу $\langle a, b \rangle$; <br>
- нет пересечений отрезков внутри лестницы; <br>
- $Q$ упорядочена по отношению $<_a$.
Часть отрезков лестницы внутри полосы будем называть ___ступеньками___.

In [12]:
class Staircase:
    """
    Структура "Лестница"

    :param Q: множество "ступенек" лестницы 
    :param a: левая граница полосы
    :param b: правая граница полосы

    """
    def __init__(self, Q, a, b):
        self.Q = Q
        self.a = a
        self.b = b

##### Определение
> Будем называть лестницу $D$ ___полной___ относительно множества отрезков $S$, если каждый отрезок из $S$ либо не охватывает полосу $\langle a, b \rangle$, либо пересекает хотя бы одну из ступенек из множества $D$.

##### Лемма (#1)
> Пусть лестница $D$ полна относительно множества отрезков $S$, где $S$ состоит из отрезков, пересекающих полосу $\langle a, b \rangle$, тогда $|S| \le Ends_{a, b}(S) + |Int(D, S)|$,<br>
где $Ends_{a, b}(S)$ это число вершин отрезков из $S$, находящихся в пределах полосы $\langle a, b \rangle$.


##### Определение
> Если точка $p$ отрезка $s$ лежит между ступеньками $i$ и $i + 1$, тогда число $i$ называется ___местоположением___ $s$ на лестнице $D$ и обозначается как $Loc(D, s)$


##### Утверждение
> Имея лестницу $D = (Q, \langle a, b \rangle)$ и множество отрезков $S$, множество $Int(D, S)$ можно найти за время $O(|S| log|Q| + |Int(D, S)|)$. <br>
Однако, если $S’$ упорядочено отношением $<_x$, где $x \in [a, b]$, тогда можно найти $Int(D, S)$ за время $O(|S| + |Q| + |Int(D, S)|)$.

## Алгоритм

---

Введем несколько дополнительных функций, чтобы упростить основной алгоритм: 

#### Split:
Функция $Split$ разделяет входное множество отрезков $L$, охватывающих некоторую полосу $\langle a, b \rangle$, на подмножества $Q$ и $L'$ так, что лестница $(Q, \langle a, b \rangle)$ полна относительно множества отрезков $L'$.

In [10]:
def Split(L, Q, L_, a, b):
    Q = []
    L_ = []

    for s in L:
        # Если отрезок s не пересекает
        # последний отрезок из Q внутри полосы <a, b>
        # то добавить в конец Q
        # иначе добавить в конец L_
        if (not Q) or (s.intersection(Q[-1]) not in (a, b)):
            Q.append(s)
        else:
            L_.append(s)

Эта функция работает за $O(|L|)$ времени.

#### Search In Strip

Зная $L$ мы можем найти $Int_{a, b}(L)$ и $R$ используя следующую рекурсивную функцию:

In [13]:
def SearchInStrip(L, R, a, b):
    Split(L, Q, L_)
    # L_ - пустое
    if not L_:
        R = Q
        return
    """Найдем Int_a,b(Q, L_)"""
    SearchInStrip(L_, R_, a, b)
    """R = merge_b(Q, R_)"""

Здесь, $Merge_x(S_1, S_2)$ это функция объединения множеств $S_1$ и $S_2$, упорядоченных по отношению $<_x$. Время выполнения $SearchInStrip$ эквивалентно сумме времён каждого её запуска. Очевидно, что время работы $i$-той функции, будет равно $O(|L_i| + |Int_{a, b}(Q_i, {L_i}')|)$, где $L_i, Q_i, {L_i}'$ это соответствующие наборы $(L_0 = L, L_{i+1} = {L_i}')$.

---

![SearchInStrip](http://images.vfl.ru/ii/1520006317/346166ef/20798448.png)
Пример выполнения функции __SearchInStrip__

---

Учитывая лемму (#1), заключаем, что функция $SearchInStrip_{a, b}(L, R)$ работает за $O(|L| + |Int_{a, b}(L)|)$.

Предположим, что все отрезки лежат в полосе $\langle a, b\rangle$. Таким образом в самом начале у нас есть пара $(S, \langle a, b\rangle)$.
Что же дальше происходит: множество $S$ ''распадается'' в подмножества $Q$ и $S'$, после чего лестница $D = (Q, \langle a, b \rangle)$ становится полной относительно множества $S'$. Необходимо найти пересечения отрезков из $D$ и $S'$, затем, все пересечения в $S'$. Чтобы найти пересечения отрезков в $S'$, мы ''режем'' полосу $\langle a, b \rangle$ и множество $S'$ по вертикале $x = c$ на полосы $\langle a, c \rangle$, $\langle c, b \rangle$ и множества $S'_{ls}$, $S'_{rs}$ соответственно, где $c$ является медианой вершин отрезков между $a$ и $b$. Затем мы рекурсивно вызываем функцию к парам $(S'_{ls}, \langle a, c \rangle)$ и $(S'_{rs}, \langle c, b \rangle)$. Ключевым является тот факт, что согласно лемме (#1) $|S'| \le Ends_{a, b}(S') + |Int(D, S')|$, таким образом, число дополнительных отрезков, появляющихся после ''разрезаний'' пропорционально числу найденных пересечений.
 

## Ключевые моменты

---

Давайте разберемся с алгоритмом более подробно:

Не умаляя общности, предположим, что все пересечения и вершины отрезков имеют разные абсциссы (в конечном счете, их можно будет отсортировать введением дополнительных свойств). Будем рассматривать целые координаты на промежутке $[1, 2N]$. Обозначим через $p_i$ слева направо конец отрезка множества $S_0$, а через $s(i)$ отрезок, которому он принадлежит соответственно.

Основная задача нашего алгоритма, это рекурсивная функция $TreeSearch$. Соединим каждый вызов функции с узлом некоего двоичного дерева (далее ''рекурсивное дерево''). Соответствующим узлом отметим все значения, множества и параметры вызова. Обозначим множество внутренних вершин за $V$.

In [ ]:
def IntersectingPairs(S):
    """
    Отсортируем 2*N вершин по координатам и
    найдем p[i], s(i), i = 1,...,2*N
    
    """
    TreeSearch(S, 1, 2*N)

In [14]:
def TreeSearch(S, a, b):
    if b - a == 1:
        """Отсортируем S по отношению <_a"""
        L = S
        SearchInStrip(L, R, a, b)
        return

    D = Staircase(Q, a, b)
    
    """
    Разделим S на Q и S_ так, что лестница
    D = Staircase(Q, a, b) будет полной, относительно множества S_

    Найдем Int(D, S_)

    """

    c = (a + b) // 2

    
    # Разделим отрезки из S_ на пересекающих
    # полосу <a, c> - S_ls и
    # полосу <c, b> - S_rs
    for s in S_:
        if spans(s, a, c) or cross(s, a, c):
            S_ls.add(s)
        if spans(s, c, b) or cross(s, c, b):
            S_rs.add(s)

    TreeSearch(S_ls, a, c)
    TreeSearch(S_rs, c, b)

Отсюда и дальше $ls(v)$, $rs(v)$ и $ft(v)$ означают, соответственно, левого сына, правого сына, и отцовскую вершину узла $v$.
Наша задача показать, что все операции с узлом $v$ происходят за $O(|S_v| + |Int(D_v, S_v')| + (a_v - b_v)logN)$, для этого возьмем во внимание, что $\sum_v |S_v| = O(N \cdot logN + K)$ (очевидно, что $\sum_v |Int(D_v, S_v')| \le K$).

Функция $TreeSearch$ похожа на функцию $SearchInStrip$. Основная разница заключается в том, что $SearchInStrip$ вызывает себя без изменения полосы, когда $TreeSearch$ делит полосу на две части, после чего рекурсивно вызывает себя для них. Другое отличие заключается в том, что множество $S_v$ не упорядочено так же, как $L$. В результате мы не можем напрямую использовать функцию $Split$ для эффективного деления $S_v$.

Чтобы решить эту проблему, представим $S_v$ как объединение трех множеств:
множества $L_v$ упорядоченного по отношению $<_a$, неупорядоченного множества $I_v$, и множества $R_v$ упорядоченного по отношению $<_b$. Расположим отрезки из $S_v$, пересекающие границу $x = a$ во множество $L_v$, отрезки пересекающие $x = b$ во множество $R_v$, и внутренние отрезки во множество $I_v$ (пример на _рисунке снизу_).

---
![pic2](http://images.vfl.ru/ii/1519671821/49495413/20745954.png)
$S_v = \{s_1, s_2, s_3, s_4, s_5\}$, $L_v = (s_1, s_3)$, $R_v = (s_4, s_3)$, $I_v = \{s_2, s_5\}$

---

Теперь мы можем вызвать функцию $Split$ для множества $L_v$ и построить $Q_v$ за $O(|L_v|) = O(|S_v|)$ времени. Но мы натыкаемся на новую проблему: передавая множества $L_v$, $I_v$ и $R_v$, необходимо найти соответствующие множества сыновей узла $v$.

Неупорядоченные множества $I_{ls(v)}$ и $I_{rs(v)}$ строятся легко. Множество $L_{ls(v)}$ будет найдено вызовом функции $Split_{a, b}(L_v, Q_v, L_{ls(v)})$ для нахождения $Int(D_v, S_v')$ в функции $TreeSearch$. Множество $L_{rs(v)}$ получается из $R_{ls(v)}$ за линейное время вставкой (если $p_c$ левая вершина отрезка) или удалением (если $p_c$ правая вершина отрезка) отрезка $s(c)$. Но как получить $R_{ls(v)}$ из $L_v$, $R_v$ и $I_v$ без сортировки?

Для листьев мы сделаем проверку вначале, и получим $R_v$ из $L_v$. Пусть $L_v$ и $I_v$ известны, и все сыновья узла $v$ - листья. Для начала запустим функцию $Split(L_v, Q_v, L_{ls(v)})$ и найдем $Q_v$ и $L_{ls(v)}$. Теперь мы должны найти $Int(D_s, S_v') = Int(D_v, L_{ls(v)}) \cup Int(D_v, I_v) \cup Int(D_v, R_{rs(v)})$, но мы не знаем $R_{rs(v)}$ и, соответственно, можем найти только $Int(D_v, L_{ls(v)}) \cup Int(D_v, I_v)$. Применим $SearchInStrip$ к множеству $L_{ls(v)}$ и получим $R_{ls(v)}$. Множество $L_{rs(v)}$ получается из $R_{ls(v)}$ вставкой или удалением отрезка $s(c)$. Применим $SearchInStrip$ к $L_{rs(v)}$ и найдем $R_{rs(v)}$. Теперь можем продолжить вычисление $Int(D_v, R_{rs(v)})$ и получим $R_v$ слиянием $Q_v$ и $R_{rs(v)}$.

Конечная функция будет выглядеть так:

In [8]:
def IntersectionPairs(S):
    """
    Отсортируем 2*N концов отрезков по абсциссе
    и найдем p[i], s(i), где i = 1, ..., 2N
    
    L = s(1)
    I = S \ ({s(1)} + {s(2)})
    
    """
    TreeSearch(L, I, 1, 2*N, R)

In [15]:
def TreeSearch(L, I, a, b, R):
    if b - a == 1:
        SearchInStrip(L, R, a, b)
        return
    Split(L, Q, L_ls, a, b)

    D = Staircase(Q, a, b)
    """Найдем Int(D, L_ls)"""

    c = (a + b) // 2

    # Разделяем отрезки из I
    # внутренние для полосы <a, c> во множество I_ls
    # внутренние для полосы <c, b> во множество I_rs
    for s in I:
        if spans(s, a, c) or cross(s, a, c):
            I_ls.add(s)
        if spans(s, c, b) or cross(s, c, b):
            I_rs.add(s)

    TreeSearch(L_ls, I_ls, a, c, R_ls)


    # Если p[c] - левый конец отрезка s(c)
    # то L_rs получается из R_ls вставкой s(c)
    # иначе L_rs получает из R_ls удалением s(c)
    if 'p[c] - левый конец отрезка s(c)':
        R_ls.remove(s(c))
    else:
        R_ls.add(s(c))
    L_rs = R_ls

    TreeSearch(L_rs, I_rs, c, b, R_rs)
    """
    Найдем Int(D, R_rs)
    
    for s in I:
        Найдем Loc(D, s) используя двоичный поиск
   
    Найдем Int(D, I) используя значения, полученные шагом выше

    R = merge_b(Q, R_rs)
    
    """

Заметим, что нахождение $Int(D_v, R_{rs(v)})$ надо делать аккуратно, так как множества $R_{rs(v)}$ и $L_{ls(v)}$ могут иметь одни и те же отрезки (которые ___пересекают___ $\langle a, b \rangle$). Мы нашли их пересечения с $D_v$ при поиске $Int(D_v, L_{ls(v)})$, и не должны вывести эти пересечения снова.

---

![](http://images.vfl.ru/ii/1520010479/bd06b5a3/20799303.png)
Пример выполнения функции __TreeSearch__. Ins. значит Insert, Del значит Delete. 

---

## Время работы

---

Для начала рассчитаем необходимую память для выполнения алгоритма. Алгоритм использует рекурсивную функцию $TreeSearch$. Последовательность вызовов функции может занять память. Эта последовательность может быть представлена как путь корня рекурсивного дерева, до узла. Соответствующий вызов этого узла занимает $O(N)$ памяти, каждый его "предок" занимает $O(|I_v| + |Q_v|)$ памяти, а остальные структуры используют $O(N)$. Очевидно, что любой путь $pt$ от корня рекурсивного дерева до какого-то узла $\sum_{v \in pt}(|I_v + |Q_v|) = O(N)(|I_v| \le b_v - a_v \le \lceil (b_{ft(v)} - a_{ft(v)})/2 \rceil)$.

В итоге для работы алгоритма требуется $O(N)$ памяти.

##### Лемма (#2)

> $\forall v \in V \  |S_v'| \le a_v - b_v + |Int(D_v, S_v')|$.

##### Доказательство

$\triangleright$ 

Утверждение напрямую вытекает из леммы (#1)
и очевидного факта, что для любого множества $S \subset S_0$, количество концов отрезков, лежащих в полосе $\langle a_v, b_v \rangle$, меньше чем  $b_v - a_v$.

$\triangleleft$

##### Теорема (#1)

> $\sum_{v \in V} |S_v'| \le 2N \lceil logN + 1 \rceil + K$

##### Доказательство

$\triangleright$

Утверждение напрямую вытекает из леммы (#2) и следующего отношения $\sum_v (b_v - a_v) \le 2N \lceil logN + 1 \rceil$.
Обозначим множество всех вершин рекурсивного дерева за $RT$.

$\triangleleft$

##### Теорема (#2)

> $\sum_{v \in RT} |S_v| \le N \lceil 4logN + 5 \rceil + 2K$

##### Доказательство

$\triangleright$

Для всех узлов, кроме корня $r$ имеет место выражение $|S_v| \le |S_{ft(v)}'|$, следовательно $\sum_{v \in RT} |S_v| \le |S_r| + \sum_{v \in RT \setminus r} |S_{ft(v)}| \le N + 2 \sum_{v \in V} |S_v'| \le N \lceil 4 logN + 5 \rceil + 2K$.

$\triangleleft$

Начальная сортировка и инициализация множеств $L_r$ и $I_r$ может быть произведена за $O(N logN)$ времени. Время работы функции $TreeSearch$ является суммой длительностей всех его вызовов. Каждый вызов от внешних узлов добавляет к этой сумме $O(|L_v| + |Int_{a, b}(L_v)|)$. Для внутренних же узлов, время требуемое для поиска $Loc(D_v, s)$ равно $O(|I_v| log N)$, а для остальных $O(|S_v| + |Int(D_v, S_v')|)$. Если мы все это сложим, то придем к выводу, что наш алгоритм работает за $O(N log^2 N + K)$. Заметим, что его скорость можно увеличить до $O(N logN + K)$, если не будем учитывать время нахождения $Loc(D_v, s)$.

Соответственно в оптимальном алгоритме Балабана $Loc(D_v, s)$ находится за $O(1)$.